In [5]:
%tb
import logging

import sys
from pyspark.sql import SparkSession

from data_transformations.citibike import distance_transformer

LOG_FILENAME = 'project.log'
APP_NAME = "Citibike Pipeline: Distance Calculation"

if __name__ == '__main__':
    logging.basicConfig(filename=LOG_FILENAME, level=logging.INFO)
    arguments = sys.argv

    # if len(arguments) != 3:
    #     logging.warning("Dataset file path and output path not specified!")
    #     sys.exit(1)

    input_dataset_path = "./resources/citibike/citibike.csv"
    output_path =  "./output"

    spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    logging.info("Application Initialized: " + spark.sparkContext.appName)
    # distance_transformer.run(spark, dataset_path, output_path)
    # logging.info("Application Done: " + spark.sparkContext.appName)

    # spark.stop()


UsageError: Line magic function `%tbimport` not found.
22/04/26 16:19:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
# Run
input_dataset = spark.read.option('header', True).csv(input_dataset_path)
input_dataset.printSchema()

root
 |-- tripduration: string (nullable = true)
 |-- starttime: string (nullable = true)
 |-- stoptime: string (nullable = true)
 |-- start station id: string (nullable = true)
 |-- start station name: string (nullable = true)
 |-- start station latitude: string (nullable = true)
 |-- start station longitude: string (nullable = true)
 |-- end station id: string (nullable = true)
 |-- end station name: string (nullable = true)
 |-- end station latitude: string (nullable = true)
 |-- end station longitude: string (nullable = true)
 |-- bikeid: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth year: string (nullable = true)
 |-- gender: string (nullable = true)



In [12]:
binput_dataset.count()

3600

In [14]:
input_dataset.rdd.getNumPartitions()

1

In [8]:
input_dataset.show(10)

+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|         364|2017-07-01 00:00:00|2017-07-01 00:06:05|             539|Metropolitan Ave ...|           40.71534825|           -73.96024116|          3107|Bedford Ave & Nas...|         40.72311651|         -73.95212324| 14744|Subscriber

In [3]:
double_cols = ['start station longitude', 'start station latitude', 'end station longitude', 'end station latitude']
input_dataset.select(double_cols)

DataFrame[start station longitude: string, start station latitude: string, end station longitude: string, end station latitude: string]

In [41]:
# Converte columns
from pyspark.sql.types import StructField, DoubleType

double_cols = ['start station longitude', 'start station latitude', 'end station longitude', 'end station latitude']
for col in double_cols:
    input_dataset = input_dataset.withColumn(col, input_dataset[col].cast(DoubleType()))